In [36]:
!pip install folium

In [4]:
import folium
import pandas as pd

In [6]:
utah_df = pd.read_csv('1871-utah-postmaster-salaries.csv')
print(utah_df.sample(5))
utah_df.dtypes


           PO_Name      County State  PM_Salary   Latitude   Longitude
66           Logan       Cache    UT        290  41.734375 -111.834666
75            Mona        Juab    UT         18  39.816621 -111.850213
76          Monroe      Sevier    UT         12  38.631639 -112.121313
45          Hebron  Washington    UT         12  37.576642 -113.819415
5   Bingham Canyon   Salt Lake    UT         12  40.541613 -112.147997


PO_Name       object
County        object
State         object
PM_Salary      int64
Latitude     float64
Longitude    float64
dtype: object

In [8]:
utah_map_empty = folium.Map(location=[40, -111], zoom_start=6)
utah_map_empty


In [10]:
def create_empty_map():
    return folium.Map(location=[40, -111], zoom_start=6)

utah_map = create_empty_map()
utah_map


In [12]:
# Check for columns with missing values
missing_values = utah_df.isna().sum()
print(missing_values)


PO_Name      0
County       0
State        0
PM_Salary    0
Latitude     4
Longitude    4
dtype: int64


In [14]:
# Filter out post offices that are missing a latitude value (ie. we don't have any location information about it)
utah_df_locations = utah_df[utah_df['Latitude'].notna()]
print(len(utah_df))
print(len(utah_df_locations))


136
132


In [16]:
folium.Marker(location=[38.41, -112.339], popup="Adamsville Post Office").add_to(utah_map)
utah_map


In [18]:
# Melanie Walsh function we will adadpt to our dataset:
# def create_map_markers(row, map_name):
#    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

def create_map_markers(row, map_name):
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['PO_Name']).add_to(map_name)


In [20]:
#create a base empty map
utah_map = create_empty_map()

#generate a random row of data
sample_row = utah_df_locations.sample(1)

#use our function on the random row
create_map_markers(sample_row, utah_map)

#display the map
utah_map


/opt/anaconda3/lib/python3.12/site-packages/folium/utilities.py:101: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/opt/anaconda3/lib/python3.12/site-packages/folium/utilities.py:107: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/opt/anaconda3/lib/python3.12/site-packages/folium/utilities.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


In [22]:
# Method 1: Using a for loop to iterate through our dataframe and add markers sequentially
# initialize an empty map
utah_map = create_empty_map()

# iterrows() allows you to loop through a dataframe row by row and return the index position + the row
for index, row in utah_df_locations.iterrows():
    print(f"Name of post office:", row[0])

#now let's iterate through and call our function for each row
for index, row in utah_df_locations.iterrows():
    create_map_markers(row, utah_map)

utah_map


Name of post office: Adamsville
Name of post office: Alma
Name of post office: Alpine City
Name of post office: American Fork
Name of post office: Bellevue
Name of post office: Bingham Canyon
Name of post office: Brigham City
Name of post office: Bullion
Name of post office: Cedar City
Name of post office: Cedar Valley
Name of post office: Centerville
Name of post office: Central City
Name of post office: Chicken Creek
Name of post office: Clarkston
Name of post office: Clifton
Name of post office: Clover Valley
Name of post office: Coalville
Name of post office: Corinne
Name of post office: Cove Creek
Name of post office: Croydon
Name of post office: Deseret
Name of post office: Diamond
Name of post office: Draper
Name of post office: Duncans Retreat
Name of post office: Echo City
Name of post office: Eden
Name of post office: Emmaville
Name of post office: Ephraim
Name of post office: Eureka
Name of post office: Fair View
Name of post office: Fairfield
Name of post office: Farmington

/var/folders/35/7877ljzn6nj5tdww9cfrpbz40000gn/T/ipykernel_98618/2508161898.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Name of post office:", row[0])


In [24]:
# Method 2: Using .apply() to add markers with our function for all rows
# initialize an empty map
utah_map = utah_map_empty

# Now apply this function to each row in our filtered DataFrame
# For each row, we'll pass:
# 1. The row itself (handled automatically by .apply())
# 2. Our map object (we need to specify this explicitly)
# 3. The "axis" value for .apply() to indicate we want to process row by row
# .apply() allows you to apply a function to each row in the dataframe
utah_df_locations.apply(
    create_map_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [26]:
# alter map appearance
def create_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=5,
                       color='red',
                       fill=True,
                       fill_color='green',
                       fill_opacity=0.5,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [30]:
# make new function to create circle markers sized by postmaster salary - this time adjusting the radius size in pixels to make it more legible
def create_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['PM_Salary']/100,
                       fill=True,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_sized_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [32]:
utah_df_locations.describe()


,PM_Salary,Latitude,Longitude
count,132.000000,132.000000,132.000000
mean,101.098485,39.907743,-112.161190
std,344.225580,1.505091,0.593117
min,4.000000,37.006375,-113.819415
25%,12.000000,38.874099,-112.379230
50%,22.000000,40.380926,-111.973830
75%,70.000000,41.080917,-111.819912
max,3600.000000,42.187500,-111.281850


In [34]:
def add_salary_buckets(salary):
    # Create a new column for the salary bucket
    if salary < 50:
        bucket = 'Low Salary'
    elif salary >= 50 and salary < 250:
        bucket = 'Medium Salary'
    elif salary >= 250 and salary < 1000:
        bucket = 'High Salary'
    else:
        bucket = 'Very High Salary'
    return bucket


In [36]:
#test out the function
add_salary_buckets(2000)


'Very High Salary'

In [38]:
utah_df_locations['Salary_Bucket'] = utah_df_locations['PM_Salary'].apply(add_salary_buckets)
utah_df_locations.head()


/var/folders/35/7877ljzn6nj5tdww9cfrpbz40000gn/T/ipykernel_98618/2959544325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utah_df_locations['Salary_Bucket'] = utah_df_locations['PM_Salary'].apply(add_salary_buckets)


,PO_Name,County,State,PM_Salary,Latitude,Longitude,Salary_Bucket
0,Adamsville,Beaver,UT,10,38.258303,-112.793835,Low Salary
1,Alma,Weber,UT,12,41.248833,-112.078275,Low Salary
2,Alpine City,Utah,UT,27,40.453283,-111.777986,Low Salary
3,American Fork,Utah,UT,130,40.375229,-111.796320,Medium Salary
4,Bellevue,Washington,UT,20,37.340815,-113.274116,Low Salary


In [40]:
# create a function to add marker sizes based on the salary bucket
def add_marker_sizes(category):
    if category == 'Low Salary':
        return 4
    elif category == 'Medium Salary':
        return 8
    elif category == 'High Salary':
        return 12
    else:
        return 16

#test out the function
add_marker_sizes('High Salary')


12

In [42]:
utah_df_locations['Marker_Size'] = utah_df_locations['Salary_Bucket'].apply(add_marker_sizes)
utah_df_locations.head(10)


/var/folders/35/7877ljzn6nj5tdww9cfrpbz40000gn/T/ipykernel_98618/3689819186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utah_df_locations['Marker_Size'] = utah_df_locations['Salary_Bucket'].apply(add_marker_sizes)


,PO_Name,County,State,PM_Salary,Latitude,Longitude,Salary_Bucket,Marker_Size
0,Adamsville,Beaver,UT,10,38.258303,-112.793835,Low Salary,4
1,Alma,Weber,UT,12,41.248833,-112.078275,Low Salary,4
2,Alpine City,Utah,UT,27,40.453283,-111.777986,Low Salary,4
3,American Fork,Utah,UT,130,40.375229,-111.796320,Medium Salary,8
4,Bellevue,Washington,UT,20,37.340815,-113.274116,Low Salary,4
5,Bingham Canyon,Salt Lake,UT,12,40.541613,-112.147997,Low Salary,4
7,Brigham City,Box Elder,UT,400,41.510213,-112.015501,High Salary,12
8,Bullion,Piute,UT,12,38.410000,-112.339000,Low Salary,4
9,Cedar City,Iron,UT,200,37.676644,-113.057171,Medium Salary,8
10,Cedar Valley,Utah,UT,13,40.327171,-112.104385,Low Salary,4


In [44]:
# make new function to create circle markers sized by salary category
def create_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['Marker_Size'],
                       fill=True,
                       opacity=0.6,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_sized_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [48]:
utah_1877_df = pd.read_csv('1877-official-register.csv')
utah_1877_df.dtypes

Name           object
State          object
Department     object
Type           object
People          int64
Latitude      float64
Longitude     float64
dtype: object

In [52]:
# Melanie Walsh function we will adadpt to our dataset:
# def create_map_markers(row, map_name):
#    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

def create_map_markers(row, map_name):
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Name']).add_to(map_name)


In [56]:
# Method 2: Using .apply() to add markers with our function for all rows
# initialize an empty map
gov_map = utah_map_empty


gov_df = utah_1877_df[utah_1877_df['Latitude'].notna()]
# Now apply this function to each row in our filtered DataFrame
# For each row, we'll pass:
# 1. The row itself (handled automatically by .apply())
# 2. Our map object (we need to specify this explicitly)
# 3. The "axis" value for .apply() to indicate we want to process row by row
# .apply() allows you to apply a function to each row in the dataframe
gov_df.apply(
    create_map_markers, # The function to apply
    map_name=gov_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

gov_map
